In [141]:
from torchvision.datasets import FashionMNIST
from torchvision import transforms
from torch.utils.data import DataLoader
from torch import nn
from torch import optim

import tqdm


In [142]:
fashion_mnist_train = FashionMNIST("data/FashionMNIST",
                                  train=True,download=True,
                                  transform=transforms.ToTensor())

In [143]:
fashion_mnist_test = FashionMNIST("data/FashionMNIST",
                                 train=False,download=True,
                                 transform=transforms.ToTensor())

In [144]:
batch_size = 128
train_loader = DataLoader(fashion_mnist_train,
                          batch_size=batch_size,shuffle=True)
test_loader = DataLoader(fashion_mnist_test,
                        batch_size=batch_size,shuffle=True)


In [145]:
class FlattenLayer(nn.Module):
    def forward(self,x):
        sizes = x.size()
        return x.view(sizes[0],-1)
    


In [146]:
conv_net = nn.Sequential(
    nn.Conv2d(1,32,5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(32),
    nn.Dropout2d(0.25),
    nn.Conv2d(32,64,5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(64),
    nn.Dropout(0.25),
    FlattenLayer()
)

In [147]:
test_input = torch.ones(1,1,28,28)
conv_output_size = conv_net(test_input).size()[-1]

In [148]:
mlp = nn.Sequential(
    nn.Linear(conv_output_size,200),
    nn.ReLU(),
    nn.BatchNorm1d(200),
    nn.Dropout(0.25),
    nn.Linear(200,10)
)

In [149]:
net = nn.Sequential(
    conv_net,
    mlp
)

In [150]:
def eval_net(net,data_loader,device="cuda:0"):
    net.eval()
    ys = []
    ypreds = []
    for x,y in data_loader:
        x = x.cuda()
        y = y.cuda()
        with torch.no_gard():
            _,y_pred = net(x).max(1)
        ys.append(y)
        ypreds.qppend(y_pred)
    ys = torch.cat(ys)
    ypreds = torch.cat(ypreds)
    
    acc = (ys == ypreds).float().sum()/len(ys)
    return acc.item()

In [151]:
def train_net(net,train_loader,optimizer_cls=optim.Adam,loss_fn=nn.CrossEntropyLoss(),n_iter=10,device="cuda:0"):
    train_losses = []
    train_acc = []
    val_acc = []
    optimizer = optim.Adam(net.parameters())
    for epoch in range(0,20):
        running_loss = 0
        net.train()
        n = 0
        n_acc = 0
        for i,(xx,yy) in tqdm.tqdmm(enumerate(train_loader),total=len(train_loader)):
            xx = xx.cuda()
            yy = yy.cuda()
            h = net(xx)
            loss = loss_fn(h,yy)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            n += len(xx)
            _,y_pred = h.max(1)
            n_acc += (yy == y_pred).float().sum().item()
            train_losses.append(running_loss/i)
            train_acc.append(n_acc/n)
            val_acc.append(eval_net(net,test_loader,device))
            print(epoch,train_losses[-1],train_acc[-1],val_acc[-1],flush=True)

In [152]:
net.cuda()
train_net(net,train_loader,test_loader,n_iter=20,device="cuda:0")

D:\python\Anaconda\envs\pytorch37\lib\site-packages\ipykernel_launcher.py:6: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  


AttributeError: module 'tqdm' has no attribute 'tqdmm'